# Analysis of `floating-car-data.xml`

In [ ]:
import pprint, math, datetime, sys, time
from time import strftime, gmtime
from lxml import etree
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

plt.rcParams["figure.figsize"] = 30, 20
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

# normal = "berlin-1pct"
# normal = "inner-berlin-output"
# normal = "normal-output"
normal = "../simulation/charlottenburg-normal"
scenarios = [
  # "apats-output"
#   "greater-berlin-1pct"
]

get_file = lambda x: f"{x}/output/floating-car-data.xml"

### Traffic counts for SUMO edges

In [ ]:
def count_vehicles_at_edges(file_path, edges, interval=60*60):
    counts = {}
    interval_counts = {}
  
    for event, element in etree.iterparse(file_path, tag="timestep"):
        time = float(element.attrib["time"])
                
        if time > 0 and time % interval == 0:
            counts[time-interval] = interval_counts
            interval_counts = {}

        for vehicle in element.getchildren():
            vid = vehicle.attrib["id"]
            edge = vehicle.attrib["lane"].split("_")[0]

            if edge not in edges:
                continue

            if "cluster" in edge:
                continue

            if edge in interval_counts:
                edge_counts = interval_counts[edge]
                if vid not in edge_counts:
                    edge_counts.append(vid)
                    interval_counts[edge] = edge_counts
            else:
                interval_counts[edge] = [vid]


        element.clear()

    return counts

In [ ]:
matsim_count_edges = ["4067952", "375311156#0", "432205808#0"]

test_count_edges = ["627460044#0"]

interval = 60*60

edges = test_count_edges

counts_per_scenario = {}
for scenario in [normal, *scenarios]:
    file = get_file(scenario)
    vehicle_counts_per_edge = count_vehicles_at_edges(file, edges=edges, interval=interval)
    counts_per_scenario[scenario] = vehicle_counts_per_edge

In [ ]:
for edge in edges:
    fig, ax = plt.subplots()
    ax.set_xticks(hours)
    ax.set_xlabel("Hour of the day", labelpad=40)
    ax.set_ylabel("No. of vehicles", labelpad=40)
    ax.set_title(f"Hourly traffic counts at edge {edge}")

    for i, scenario in enumerate([normal, *scenarios]):
        vehicle_counts_per_edge = counts_per_scenario[scenario]
        hours = list(map(lambda i: i / interval, vehicle_counts_per_edge.keys()))

        counts = []
        for edge_counts in vehicle_counts_per_edge.values():
            value = 0
            if edge in edge_counts:
                value = len(edge_counts[edge])
            counts.append(value)

        offset = 0.2
        a = ax.bar(np.array(hours) + ((i - 1) * offset), counts, width=offset, label=scenario)

    ax.legend()
fig.savefig(f"traffic-count.png", bbox_inches="tight")


### Traffic heatmap

In [ ]:
from pyproj import Proj, transform
from shapely.geometry import Point

external_crs = "EPSG:4326"
internal_crs = "EPSG:3068"

extProj = Proj(external_crs)
inProj = Proj(internal_crs)

def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def count_vehicles_in_grid(file_path, bbox, start=0, end=math.inf):
    lat_min, long_min, lat_max, long_max = bbox

    ymin, xmin = transform(extProj, inProj, lat_min, long_min)
    ymax, xmax = transform(extProj, inProj, lat_max, long_max)

    print(xmin, xmax)
    print(ymin, ymax)

    cellsize = 10

    xgrid = np.linspace(xmin, xmax, int((xmax - xmin) / cellsize))
    ygrid = np.linspace(ymin, ymax, int((ymax - ymin) / cellsize))

    x_points = gpd.GeoDataFrame({"geometry": [Point(x, ygrid[0]) for x in xgrid]})
    y_points = gpd.GeoDataFrame({"geometry": [Point(xgrid[0], y) for y in ygrid]})

    x_points.crs = internal_crs
    y_points.crs = internal_crs

    long_points = x_points.to_crs(external_crs)
    lat_points = y_points.to_crs(external_crs)

    long_grid = np.array(long_points.geometry.x)
    lat_grid = np.array(lat_points.geometry.y)
    
    vehicles_per_cell = {}
    skip_timestep = False
    counts = np.zeros((len(lat_grid), len(long_grid)))
    
    for event, element in etree.iterparse(file_path, tag="timestep"):
        t = float(element.attrib["time"])

        if t < start:
            continue

        if t > end:
            break

        for vehicle in element.getchildren():
            vid = vehicle.attrib["id"]
            long = float(vehicle.attrib["x"])
            lat = float(vehicle.attrib["y"])

            cell_long = find_nearest(long_grid, long)
            cell_lat = find_nearest(lat_grid, lat)
            cell_key = f"{cell_long},{cell_lat}"

            if cell_key in vehicles_per_cell:
                if vid in vehicles_per_cell[cell_key]:
                    continue

            i = np.nonzero(lat_grid == cell_lat)[0][0]
            j = np.nonzero(long_grid == cell_long)[0][0]
            counts[i][j] += 1

            if cell_key in vehicles_per_cell:
                vehicles_per_cell[cell_key].append(vid)
            else:
                vehicles_per_cell[cell_key] = [vid]

        element.clear()

    return (counts, [xmin, xmax, ymin, ymax])

In [ ]:
bbox = [52.4828, 13.2136, 52.5330, 13.3594]

counts_per_scenario = {}


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(60, 40)



file = get_file(normal)
counts, extent = count_vehicles_in_grid(file, bbox)
img = ax.imshow(counts, origin="lower", cmap="Reds", extent=extent)
img.cmap.set_under("w")
img.set_clim(0.1)

# file = get_file("apats-output")
# counts = count_vehicles_in_grid(file, bbox)
# img = ax2.imshow(counts, origin="lower", cmap="Reds", extent=[xmin, xmax, ymin, ymax])
# img.cmap.set_under("w")
# img.set_clim(0.1)

fig.savefig("heatmap.png", bbox_inches="tight")
plt.close()

In [ ]:
counts = count_vehicles_in_grid(s_file)

fig, ax = plt.subplots()
img = ax.imshow(counts, origin="lower", cmap="viridis_r", extent=[xmin, xmax, ymin, ymax])
img.cmap.set_under("w")
img.set_clim(0.1)